In [12]:
import pandas as pd

from tqdm import tqdm

import transformers
import torch
import torchmetrics
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.tuner import Tuner
import wandb
from train import Dataset, Dataloader, Model

In [17]:
model_name="snunlp/KR-ELECTRA-discriminator"
batch_size=16
max_epoch=10
shuffle=True
learning_rate=2e-5
train_path='~/data/train_resampled_swap_v2.csv'
dev_path='~/data/dev.csv'
test_path='~/data/dev.csv'
predict_path='~/data/dev.csv'
weight_decay=0.01
warm_up_ratio=0.3
loss_func="MSE"
run_name="001"
project_name="STS_snunlp_9250"
eda=True

In [15]:
def get_int_prediction(model_path, model_name, batch_size, shuffle, train_path, dev_path, test_path, predict_path):
    # dataloader와 model을 생성합니다.
    dataloader = Dataloader(model_name, batch_size, shuffle, train_path, dev_path, test_path, predict_path)

    # gpu가 없으면 accelerator='cpu', 있으면 accelerator='gpu'
    trainer = pl.Trainer(accelerator='gpu')

    # Inference part
    # 저장된 모델로 예측을 진행합니다.
    if model_path.endswith(".pt"):
        model = torch.load(model_path)
    elif model_path.endswith(".ckpt"):
        model = Model.load_from_checkpoint(model_path)
    predictions = torch.cat(trainer.predict(model=model, datamodule=dataloader)).round().long()
    return predictions

In [ ]:
    # 예측된 결과를 형식에 맞게 반올림하여 준비합니다.
    predictions = list(float(i) for i in torch.cat(predictions))

    # output 형식을 불러와서 예측된 결과로 바꿔주고, output.csv로 출력합니다.
    output = pd.read_csv('~/data/sample_submission.csv')
    output['target'] = predictions
    output.to_csv('output1.csv', index=False)

In [16]:
dev = pd.read_csv(dev_path)
dev.head()

,id,source,sentence_1,sentence_2,label,binary-label
0,boostcamp-sts-v1-dev-000,nsmc-sampled,액션은개뿔 총몇번쏘고 끝입니다,액션은 흉내만 내고 그마저도 후반부에는 슬로우모션 처리,2.0,0.0
1,boostcamp-sts-v1-dev-001,slack-rtt,감격스러워 입막으심?,너무 감동해서 입 다물어?,3.4,1.0
2,boostcamp-sts-v1-dev-002,nsmc-rtt,이번 년도에 본 영화 중 가장 최악의 영화......,올해 본 영화 중 최악...,4.0,1.0
3,boostcamp-sts-v1-dev-003,slack-rtt,특히 평소 뮤직채널에 많은 영감을 불어넣어주시는!,"특히, 당신은 항상 많은 음악 채널에 영감을 줍니다!",3.4,1.0
4,boostcamp-sts-v1-dev-004,slack-sampled,다음 밥스테이지가 기대됩니다~ ㅎ,다음 후기도 기대됩니다~~,1.4,0.0


In [18]:
df = dev.copy()
df['label_class'] = df.label.round(0).astype(int)

In [19]:
df.head()

,id,source,sentence_1,sentence_2,label,binary-label,label_class
0,boostcamp-sts-v1-dev-000,nsmc-sampled,액션은개뿔 총몇번쏘고 끝입니다,액션은 흉내만 내고 그마저도 후반부에는 슬로우모션 처리,2.0,0.0,2
1,boostcamp-sts-v1-dev-001,slack-rtt,감격스러워 입막으심?,너무 감동해서 입 다물어?,3.4,1.0,3
2,boostcamp-sts-v1-dev-002,nsmc-rtt,이번 년도에 본 영화 중 가장 최악의 영화......,올해 본 영화 중 최악...,4.0,1.0,4
3,boostcamp-sts-v1-dev-003,slack-rtt,특히 평소 뮤직채널에 많은 영감을 불어넣어주시는!,"특히, 당신은 항상 많은 음악 채널에 영감을 줍니다!",3.4,1.0,3
4,boostcamp-sts-v1-dev-004,slack-sampled,다음 밥스테이지가 기대됩니다~ ㅎ,다음 후기도 기대됩니다~~,1.4,0.0,1
